## let's build our final model

we uploaded on one hand two sets of spectrograms : The original unaltered wav to mel spectrogram. and the same set with random frequence masking, on the other hand, four folders with the same wav augmented : shifting, noisy, fade in n out, and dimmed versions of the .wavs

In [1]:
from keras import Sequential, Input, layers
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split

from keras.utils import to_categorical

2025-12-16 11:18:55.704861: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import numpy as np
import pandas as pd
import os
import librosa

let's import our csv with our files name and notes (y)

In [3]:
sample_df = pd.read_csv("/home/edgar/code/Arkyed/08-Project/databass-code/DataBass/data/preprocessed/instruments_list_augmented.csv")

sample_df.sample()

/tmp/ipykernel_140327/2145269328.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  sample_df = pd.read_csv("/home/edgar/code/Arkyed/08-Project/databass-code/DataBass/data/preprocessed/instruments_list_augmented.csv")


,fileID,instrument,instrumentsetting,playstyle,midinr,string,fret,fxgroup,fxtype,fxsetting,filenr,note_name,file_path
87199,NaN,B,2,3,53,4,10,1,12,2,12790,F3,/home/edgar/code/Arkyed/08-Project/databass-co...


In [4]:
sample_augmented_df = sample_df[41184:]

In [5]:
sample_augmented_df

,fileID,instrument,instrumentsetting,playstyle,midinr,string,fret,fxgroup,fxtype,fxsetting,filenr,note_name,file_path
41184,NaN,B,3,3,45,3,7,4,42,3,9758,A2,/home/edgar/code/Arkyed/08-Project/databass-co...
41185,NaN,G,8,1,54,3,4,4,42,2,40279,Fx3,/home/edgar/code/Arkyed/08-Project/databass-co...
41186,NaN,B,1,2,30,1,2,2,23,3,16279,F#1,/home/edgar/code/Arkyed/08-Project/databass-co...
41187,NaN,B,1,2,44,2,11,3,31,3,1949,G#2,/home/edgar/code/Arkyed/08-Project/databass-co...
41188,NaN,G,9,3,72,6,8,1,12,2,28076,C5,/home/edgar/code/Arkyed/08-Project/databass-co...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
205915,NaN,G,9,3,76,6,12,2,23,1,25584,E5,/home/edgar/code/Arkyed/08-Project/databass-co...
205916,NaN,G,7,3,50,1,10,3,34,1,33941,D3,/home/edgar/code/Arkyed/08-Project/databass-co...
205917,NaN,G,6,1,67,5,8,2,23,2,25645,G4,/home/edgar/code/Arkyed/08-Project/databass-co...
205918,NaN,G,8,1,73,6,9,4,41,2,38451,Cx5,/home/edgar/code/Arkyed/08-Project/databass-co...


In [6]:
augmentation_types = ['dimmed', 'noisy', 'shifted', 'fade_in_out']
base_augmented_path = '/home/edgar/code/Arkyed/08-Project/databass-code/DataBass/data'  # TODO: Replace with actual path

augmented_folders = {
    aug_type: os.path.join(base_augmented_path, aug_type)
    for aug_type in augmentation_types
}
augmented_folders

{'dimmed': '/home/edgar/code/Arkyed/08-Project/databass-code/DataBass/data/dimmed',
 'noisy': '/home/edgar/code/Arkyed/08-Project/databass-code/DataBass/data/noisy',
 'shifted': '/home/edgar/code/Arkyed/08-Project/databass-code/DataBass/data/shifted',
 'fade_in_out': '/home/edgar/code/Arkyed/08-Project/databass-code/DataBass/data/fade_in_out'}

In [ ]:
augmented_file_paths = []
augmented_labels = []

#run this code in chunk as to not overload the memory:


for aug_type, folder_path in augmented_folders.items():
    if os.path.exists(folder_path):
        print(f"Loading augmented data from: {aug_type}")
        for file in sorted(os.listdir(folder_path)):
            if file.endswith(".wav"):
                # Look up label in CSV
                matching_row = sample_augmented_df[sample_augmented_df['file_path'].str.split('/').str[-1] == file]  # adjust column name if needed
                if not matching_row.empty:
                    label = matching_row.iloc[0]['note_name'].replace('#','sharp')  # or 'label' column
                    full_path = os.path.join(folder_path, file)

                    augmented_file_paths.append(full_path)
                    augmented_labels.append(label)
                else:
                    print(f"Warning: {file} not found in CSV")
print(f"Total augmented files loaded: {len(augmented_file_paths)}")

In [8]:
def generate_mel_spectrogram(y, sr, fmax=8000, n_fft=2048,
                             normalize='minmax', target_shape=(128, 128), duration=2.0):
    """
    generate a mel-spectrogram from y, sr extracted from the audio file.

    Args:
        y (np nd.array): the audio file extracted with librosa
        sr (int): the sampling rate of the audio file
        n_mels (int): number of Mel. bands. Default: 128.
        fmax (int): max frequency to use. Default: 8000 Hz.
        n_fft (int): size of the FFT window (higher = better frequencial resolution)
        normalize (bool or str): type of normalization you want to perform:
            False = no normalization, 'minmax' = MinMax normalization,
            'standard' = standard normalization
        target_shape (tuple): target size (height * width). default (128, 128)
        duration (float): duration of the audio file to analyze. default 2 sec

    Returns:
        np.ndarray: Mel-spectrogram (shape: [n_mels, time_steps]).
    """
    # Compute hop_length to get target size
    n_mels, width_target = target_shape
    # step between windows
    hop_length = int((duration * sr) / (width_target - 1))


    # create the Mel-spectrogram
    mel_spectrogram = librosa.feature.melspectrogram(
        y=y,
        sr=sr,
        n_mels=n_mels,
        fmax=fmax,
        n_fft=n_fft,
        hop_length=hop_length
    )

    # dB conversion
    mel_spectrogram_db = librosa.power_to_db(mel_spectrogram, ref=np.max)

    if normalize==False:
        return mel_spectrogram_db

    elif normalize=='minmax':
        mel_spectrogram_db_minmax = (mel_spectrogram_db - mel_spectrogram_db.min()) / (mel_spectrogram_db.max() - mel_spectrogram_db.min())
        return mel_spectrogram_db_minmax

    elif normalize=='standard':
        mel_spectrogram_db_standard = (mel_spectrogram_db - mel_spectrogram_db.mean()) / mel_spectrogram_db.std()
        return mel_spectrogram_db_standard

In [9]:
augmented_file_paths[0].split('/')[-1].count('B') , augmented_file_paths[0].split('/')[-1]

(1, 'dimmed_B11-28100-1111-00001.wav')

In [ ]:
#link the file to the labels
augmented_data = zip(augmented_file_paths, augmented_labels)

#for each file, export a mel_spectrogram and place it in the note appropriate folder
for file_path, label in augmented_data: #test 10
    y, sr = librosa.load(file_path, sr=None, duration=2.0)
    mel_spectrogram = generate_mel_spectrogram(y, sr, target_shape=(128, 128), duration=2.0)
    #check if file name conatains 'G' means it's a guitar note to be stored in spectrograms/G, if contains 'B' it's a bass note to be stored in spectrograms/B
    if 'G' in file_path.split('/')[-1]:
        instrument_folder = 'G'
    elif 'B' in file_path.split('/')[-1]:
        instrument_folder = 'B'

    # Create directory for the label if it doesn't exist
    label_dir = os.path.join('/home/edgar/code/Arkyed/08-Project/databass-code/DataBass/data/spectrograms', instrument_folder, label)
    os.makedirs(label_dir, exist_ok=True)

    # Save the mel-spectrogram as a .npy file
    base_filename = os.path.splitext(os.path.basename(file_path))[0]
    save_path = os.path.join(label_dir, f"{base_filename}_mel.npy")
    np.save(save_path, mel_spectrogram)

In [ ]:
path_data = data_path

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import Sequence, to_categorical
from tensorflow.keras.callbacks import EarlyStopping

file_paths = []
labels = []
for label in sorted(os.listdir(path_data)):
    label_path = os.path.join(path_data, label)
    if os.path.isdir(label_path):
        print(f"Indexing label: {label}")
        for file in sorted(os.listdir(label_path)):
            if file.endswith(".npy"):
                file_paths.append(os.path.join(label_path, file))
                labels.append(label)

print(f"Indexed {len(file_paths)} spectrogram files.")


In [ ]:
layers.GlobalAveragePooling2D()

In [ ]:
def build_model(hp):
    input_shape=(256, 256, 1)
    num_classes=28
    learning_rate=0.001
    model = keras.models.Sequential()
    # Couches de convolution
    model.add(layers.Conv2D(filters=hp.Int("units", min_value=32, max_value=512, step=32),kernel_size=(5, 5), padding='same', activation='relu', input_shape=input_shape))
    model.add(layers.Dropout(hp.Float('dropout', min_value=0.1, max_value=0.3, step=0.1)))
    if hp.Boolean('MaxPool'):
        model.add(layers.MaxPooling2D())

    model.add(layers.Conv2D(filters=hp.Int("units", min_value=32, max_value=512, step=32),kernel_size=(3, 3), padding='same', activation=hp.Choice("activation", ["relu", "tanh"])))
    if hp.Boolean('MaxPool'):
        model.add(layers.MaxPooling2D((2, 2)))

    # Aplatissement et couches denses
    model.add(layers.GlobalAveragePooling2D())
    if hp.Boolean('Dropout1'):
        model.add(layers.Dropout(0.2))
    model.add(layers.Dense(hp.Int("units", min_value=32, max_value=512, step=32), activation='relu'))
    if hp.Boolean('Dropout2'):
        model.add(layers.Dropout(0.3))
    model.add(layers.Dense(num_classes, activation='softmax'))

    # Compilation
    if hp.Choice('optimizer', ['adam', 'rmsprop']) == 'adam':
        optimizer = Adam(learning_rate=learning_rate)
    elif hp.Choice('optimizer', ['adam', 'rmsprop']) == 'rmsprop':
        optimizer = RMSprop(learning_rate=learning_rate)

    model.compile(
        optimizer=optimizer,
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

In [ ]:
le = LabelEncoder()
le.fit(labels)
encoded_labels = le.transform(labels)
num_classes = len(le.classes_)
print(f"Number of classes: {num_classes}")
X_train, X_val, y_train, y_val = train_test_split(
    file_paths, encoded_labels, test_size=0.2, stratify=encoded_labels, random_state=42
)
batch_size = 32
train_sequence = SpectrogramSequence(X_train, y_train, batch_size, num_classes)
val_sequence = SpectrogramSequence(X_val, y_val, batch_size, num_classes)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(
    train_sequence,
    validation_data=val_sequence,
    epochs=50,
    callbacks=[early_stopping]
)

model.save('/kaggle/working/databass_kaggle.keras')
import pickle
with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(le, f)

In [13]:
sample_list = sample_df['file_path'].split('/')[-1]

for i in range()


SyntaxError: expected ':' (3865897108.py, line 3)

In [2]:
import keras_tuner
import keras
from keras import Sequential, Input, layers
from keras.optimizers import Adam, RMSprop
from keras.callbacks import EarlyStopping, ModelCheckpoint

2025-12-16 13:24:14.431463: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
def build_model(hp):
    input_shape=(256, 256, 1)
    num_classes=28
    learning_rate=0.001
    model = keras.models.Sequential()
    # Couches de convolution
    model.add(layers.Conv2D(filters=hp.Int("units", min_value=32, max_value=512, step=32),kernel_size=(5, 5), padding='same', activation='relu', input_shape=input_shape))
    model.add(layers.Dropout(hp.Float('dropout', min_value=0.1, max_value=0.3, step=0.1)))
    if hp.Boolean('MaxPool'):
        model.add(layers.MaxPooling2D())

    model.add(layers.Conv2D(filters=hp.Int("units", min_value=32, max_value=512, step=32),kernel_size=(3, 3), padding='same', activation=hp.Choice("activation", ["relu", "tanh"])))
    if hp.Boolean('MaxPool'):
        model.add(layers.MaxPooling2D((2, 2)))

    # Aplatissement et couches denses
    layers.GlobalAveragePooling2D(),
    if hp.Boolean('Dropout1'):
        model.add(layers.Dropout(0.2))
    model.add(layers.Dense(hp.Int("units", min_value=32, max_value=512, step=32), activation='relu'))
    if hp.Boolean('Dropout2'):
        model.add(layers.Dropout(0.3))
    model.add(layers.Dense(num_classes, activation='softmax'))

        # Compilation
    if hp.Choice('optimizer', ['adam', 'rmsprop']) == 'adam':
        optimizer = Adam(learning_rate=learning_rate)
    elif hp.Choice('optimizer', ['adam', 'rmsprop']) == 'rmsprop':
        optimizer = RMSprop(learning_rate=learning_rate)

    model.compile(
        optimizer=optimizer,
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

model = build_model(keras_tuner.HyperParameters())
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_1 (Conv2D)               │ (None, 256, 256, 32)   │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256, 256, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 256, 256, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256, 256, 32)   │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256, 256, 28)   │           924 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,060 (47.11 KB)

 Trainable params: 12,060 (47.11 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=3,
    executions_per_trial=2,
    overwrite=True,
    directory="my_dir",
    project_name="helloworld",
)

y = sample_df['note']
y_cat = to_categorical(y)